**Name:** \_\_\_\_\_Huanchen Wang\_\_\_\_\_

**EID:** \_\_\_\_\_57558749\_\_\_\_\_

**Kaggle Team Name:** \_\_\_\_\_unknownwang\_\_\_\_\_

# CS5489 - Assignment 1 - Movie Rating Prediction

## Final submission
In this file, put the code that generates your final Kaggle submission. It will be used to verify that your Kaggle submission is reproducible.

In [29]:
%matplotlib inline
import matplotlib_inline   # setup output image format
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
import csv
random.seed(100)

In [30]:
def read_text_data(fname):
    txtdata = []
    revid   = []
    classes = []
    with open(fname, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            # get the text
            txtdata.append(row[0])
            revid.append(row[1])
            # get the class (convert to integer)
            if len(row)>2:
                classes.append(int(row[2]))
    
    if (len(classes)>0) and (len(txtdata) != len(revid)):        
        warn.error("mismatched length!")
    
    return (txtdata, revid, classes)

def write_csv_kaggle_sub(fname, Y):
    # fname = file name
    # Y is a list/array with class entries
    
    # header
    tmp = [['Id', 'Prediction']]
    
    # add ID numbers for each Y
    for (i,y) in enumerate(Y):
        tmp2 = [(i+1), y]
        tmp.append(tmp2)
        
    # write CSV file
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(tmp)

In [56]:
# load the data
(traintxt, trainrevid, trainY) = read_text_data("movierating_train.txt")
(testtxt, testrevid, _)        = read_text_data("movierating_test.txt")

print(len(traintxt))
print(len(testtxt))

4006
1000


In [ ]:
### INSERT YOUR CODE HERE

In [57]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
# nltk.download()
import datasets

In [58]:
def tokenize_word(doc):
    # get stopwords list
    stop_words = stopwords.words('english')
    # tokenize the text each word initially
    tokens = nltk.word_tokenize(doc)
    # remove the punctuation and stopwords in the text
    tokens = [word for word  in tokens if word not in stop_words and word not in punctuation]
    return ' '.join(tokens)

traintxt = [tokenize_word(doc) for doc in traintxt]
testtxt = [tokenize_word(doc) for doc in testtxt]

dataset= datasets.Dataset.from_dict({'text':traintxt, 'label':trainY})


In [34]:
# from datasets import load_metric
import torch
use_gpu = torch.cuda.is_available()
import evaluate
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [59]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True,max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

loading configuration file config.json from cache at C:\Users\huancwang2/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at C:\Users\huancwang2/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\voc

  0%|          | 0/5 [00:00<?, ?ba/s]

In [60]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [61]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",
                                                            num_labels=4)
training_args = TrainingArguments(output_dir="test_trainer", learning_rate=2e-5, per_device_train_batch_size=8, per_device_eval_batch_size=8, seed=46, 
                                  lr_scheduler_type='cosine', logging_dir="test_trainer", evaluation_strategy='epoch', metric_for_best_model='accuracy', greater_is_better=True)

loading configuration file config.json from cache at C:\Users\huancwang2/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache

In [62]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [63]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
d:\Anaconda\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3605
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1353
  Number of trainable parameters = 108313348


  0%|          | 0/1353 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 401
  Batch size = 8


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 1.2587769031524658, 'eval_accuracy': 0.32418952618453867, 'eval_runtime': 5.7466, 'eval_samples_per_second': 69.78, 'eval_steps_per_second': 8.875, 'epoch': 1.0}


...

Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 401
  Batch size = 8


  0%|          | 0/51 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.9391842484474182, 'eval_accuracy': 0.6034912718204489, 'eval_runtime': 6.1126, 'eval_samples_per_second': 65.602, 'eval_steps_per_second': 8.343, 'epoch': 3.0}
{'train_runtime': 484.6037, 'train_samples_per_second': 22.317, 'train_steps_per_second': 2.792, 'train_loss': 1.0552321248643062, 'epoch': 3.0}


TrainOutput(global_step=1353, training_loss=1.0552321248643062, metrics={'train_runtime': 484.6037, 'train_samples_per_second': 22.317, 'train_steps_per_second': 2.792, 'train_loss': 1.0552321248643062, 'epoch': 3.0})

In [75]:
testdataset = datasets.Dataset.from_dict({'text':testtxt})

test_tokenized_datasets = testdataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [77]:
test_labels = trainer.predict(test_tokenized_datasets).predictions.argmax(-1)
test_labels

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


  0%|          | 0/125 [00:00<?, ?it/s]

array([2, 1, 2, 2, 2, 2, 2, 0, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1,
       0, 2, 2, 1, 0, 0, 3, 2, 2, 3, 2, 1, 2, 2, 0, 1, 2, 2, 2, 1, 2, 2,
       0, 1, 3, 1, 2, 0, 1, 1, 2, 2, 2, 3, 1, 1, 1, 1, 3, 0, 2, 1, 2, 2,
       1, 2, 2, 1, 2, 2, 0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 3, 0, 1, 2,
       2, 3, 0, 1, 1, 1, 0, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2,
       1, 1, 1, 2, 1, 3, 1, 2, 1, 1, 1, 2, 1, 3, 2, 3, 2, 2, 2, 2, 3, 1,
       0, 2, 2, 3, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 1, 1,
       0, 3, 1, 1, 2, 1, 1, 0, 3, 2, 1, 0, 1, 2, 2, 1, 1, 1, 2, 1, 2, 3,
       1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1,
       1, 1, 1, 0, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 0, 1, 2, 2,
       2, 0, 1, 0, 3, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 0, 2, 2, 2, 1, 2, 1,
       2, 2, 2, 2, 2, 3, 0, 2, 2, 2, 1, 2, 2, 3, 0, 2, 3, 3, 3, 2, 1, 2,
       2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 3, 3, 2, 1, 2, 0, 1, 2, 1, 2, 2, 1,
       1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 3, 2, 1, 1,

In [78]:
test_labels = test_labels.tolist()
write_csv_kaggle_sub("my_submission.csv", test_labels)